In [ ]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

arnav1605_tomato_data_path = kagglehub.dataset_download('arnav1605/tomato-data')

print('Data source import complete.')


In [ ]:
import os
import torch
import torch.nn as nn
import torchvision
from torchvision import transforms, datasets
from torch.utils.data import DataLoader, random_split, Subset
from torchmetrics.classification import Accuracy
import torchmetrics
import torchvision.models as models

import pandas as pd

from sklearn.model_selection import KFold

device= torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
train_transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.RandomHorizontalFlip(),  #Removed RandomCrop(32) as it resized the image back
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])   #pretrained ImageNet values
])

test_transform= transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

In [ ]:
path= "/kaggle/input/tomato-data/data/Training_set"

In [ ]:
full_dataset = datasets.ImageFolder(root=path, transform=train_transform)

In [ ]:
model = models.densenet121(weights=models.DenseNet121_Weights.DEFAULT)
num_features = model.classifier.in_features
model.classifier = nn.Sequential(
    nn.Linear(num_features, 512),
    nn.ReLU(),
    nn.Dropout(0.3),
    nn.Linear(512, 10)
)
model = model.to(device)

In [ ]:
def train(dataloader, model, loss_function, optimizer):
  model.train()
  total_loss= 0

  for batch, (image, label) in enumerate(dataloader):
    image= image.to(device)
    label= label.to(device)

    prediction= model(image)
    loss= loss_function(prediction, label)

    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

    total_loss+= loss

  avg_loss= total_loss / len(dataloader)
  print(f"Training Average Loss: {avg_loss:.4f}")

In [ ]:
val_accuracy = Accuracy(task="multiclass", num_classes=10).to(device)

def validate(dataloader, model, loss_function):
    model.eval()
    total_loss = 0
    val_accuracy.reset()

    with torch.no_grad():
        for image, label in dataloader:
            image = image.to(device)
            label = label.to(device)

            pred = model(image)
            loss = loss_function(pred, label)
            total_loss += loss

            val_accuracy.update(pred, label)

    avg_loss = total_loss / len(dataloader)
    accuracy = val_accuracy.compute() * 100  # Convert to %

    print(f"Validation Loss: {avg_loss:.4f}, Validation Accuracy: {accuracy:.2f}%")

    return accuracy


In [ ]:
test_accuracy = Accuracy(task="multiclass", num_classes=10).to(device)

def test(dataloader, model, loss_function):
    model.eval()
    total_loss = 0
    test_accuracy.reset()   #Reset metric state for each epoch

    with torch.no_grad():
        for image, label in dataloader:
            image = image.to(device)
            label = label.to(device)

            pred = model(image)
            loss = loss_function(pred, label)
            total_loss += loss

            test_accuracy.update(pred, label)

    avg_loss = total_loss / len(dataloader)
    accuracy = test_accuracy.compute() * 100  # Convert to %

    print(f"Test Loss: {avg_loss:.4f}, Test Accuracy: {accuracy:.2f}%")

    return accuracy

In [ ]:

k_folds = 5
epochs = 50
kfold = KFold(n_splits=k_folds, shuffle=True, random_state=42)

for fold, (train_idx, val_idx) in enumerate(kfold.split(full_dataset)):
    print(f"\n---------Fold {fold + 1}-----------")

    # Create data subsets and loaders
    train_subset = Subset(full_dataset, train_idx)
    val_subset = Subset(full_dataset, val_idx)

    train_loader = DataLoader(train_subset, batch_size=32, shuffle=True, num_workers=2, pin_memory=True)
    val_loader = DataLoader(val_subset, batch_size=32, shuffle=False, num_workers=2, pin_memory=True)


    loss_function = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', patience=3, verbose=True)

    best_val_acc = 0.0

    for epoch in range(epochs):
        print(f"\nEpoch {epoch + 1}/{epochs}")
        train(train_loader, model, loss_function, optimizer)
        val_acc = validate(val_loader, model, loss_function)
        scheduler.step(val_acc)

        # Saving best model per fold
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save(model.state_dict(), f"best_model_fold{fold+1}.pth")

    print(f"\nBest Validation Accuracy in Fold {fold + 1}: {best_val_acc:.2f}%")
    print("---------------------------------------------------------------\n")
